Data Analysis of OD-Measurment data. 
File-type - xslx, not CSV. 
Values are in a table

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import re
from scipy import stats
import wx
import statistics
import json
import uuid
from functools import reduce


GUI for Plate layout, etc. 

In [ ]:
blank = False
global blank_dict
blank_dict = {}
key_list = {}
nested_dict = {}
wells_clicked = []
blank_measurements = []
new_keys = []
media_selection = []
status_list = []
file_name = None
Evolved = None
global key_wells
key_wells = {}
global unique_key
unique_key = f"{uuid.uuid4()}"

user_input = pd.DataFrame(columns = ['Strain', 'Wells', 'Media', 'Status'])

class PlateFrame(wx.Frame):
    def __init__(self, parent, title, size=(800, 600)):
        super(PlateFrame, self).__init__(parent, title=title, size=size)
        self.CenterOnScreen(wx.BOTH)
        self.initUI()

    def initUI(self):
        panel = wx.Panel(self)
        main_sizer = wx.BoxSizer(wx.VERTICAL)

        # Strain ID input
        strain_sizer = wx.BoxSizer(wx.HORIZONTAL)
        strain_label = wx.StaticText(panel, label="Enter Bacterial Strain:")
        self.strain_text = wx.TextCtrl(panel, style=wx.TE_PROCESS_ENTER)
        strain_sizer.Add(strain_label, 0, wx.ALL | wx.ALIGN_CENTER_VERTICAL, 5)
        strain_sizer.Add(self.strain_text, 1, wx.EXPAND | wx.ALL, 5)

        #File path button
        self.file_button = wx.Button(panel, label = 'Raw-data file')
        main_sizer.Add(self.file_button, wx.ALIGN_CENTER_HORIZONTAL | wx.ALL, 5)
        self.file_button.Bind(wx.EVT_BUTTON, self.get_file_path)

        main_sizer.Add((-1, 10))

        #Add evolved / non evolved Checkmark:
        status_selection = ['Evolved', 'Non-evolved', 'BLANK']
        self.status_button = wx.ComboBox(panel, choices = status_selection, style = wx.CB_DROPDOWN)
        main_sizer.Add(self.status_button, 0, wx.EXPAND|wx.ALL, 10)

        #Add media selection --> Drop down with pre selection or new entry
        media_selection = ['AAM + HFD', 'AAM + CHOW', 'Caecum Media', 'Standard AAM']
        self.media_button = wx.ComboBox(panel, choices = media_selection, style = wx.CB_DROPDOWN)
        self.media_button.Bind(wx.EVT_COMBOBOX, self.on_media_selection)
        main_sizer.Add(self.media_button, 0, wx.EXPAND|wx.ALL, 10)

        #Add Incoulation time button
        inoc_time = ['24h inoculation', 'Diet change inoculation']
        self.inoc_button = wx.ComboBox(panel, choices = inoc_time, style = wx.CB_DROPDOWN)
        main_sizer.Add(self.inoc_button, 0, wx.EXPAND|wx.ALL, 10)

        # Add Strain-Well button
        self.add_button = wx.Button(panel, label='Add Strain-Well')
        strain_sizer.Add(self.add_button, 0, wx.ALL, 5)
        self.add_button.Bind(wx.EVT_BUTTON, self.on_add_button_click)

        main_sizer.Add(strain_sizer, 0, wx.EXPAND | wx.ALL, 5)

        # Space between strain input and wells
        main_sizer.Add((-1, 10))

        # Wells
        well_sizer = wx.GridSizer(8, 12, 5, 5)
        for row in range(8):
            for col in range(12):
                well_button = wx.Button(panel, id=wx.ID_ANY, label=f'{chr(65 + row)}{col + 1}', size=(60, 60),
                                         style=wx.BU_EXACTFIT | wx.BORDER_NONE)
                well_button.Bind(wx.EVT_BUTTON, self.onWellClick)
                well_button.SetBackgroundColour(wx.Colour(255, 255, 255))
                well_button.SetFont(wx.Font(11, wx.FONTFAMILY_DEFAULT, wx.FONTSTYLE_NORMAL, wx.FONTWEIGHT_NORMAL))
                well_button.SetWindowStyleFlag(wx.BORDER_NONE)
                well_sizer.Add(well_button, 0, wx.EXPAND)
        main_sizer.Add(well_sizer, 1, wx.EXPAND | wx.ALL, 5)

        # Space below the wells
        main_sizer.Add((-1, 20))

        self.button = wx.Button(panel, label = 'BLANK measurement')
        main_sizer.Add(self.button, 0, wx.ALIGN_CENTER_HORIZONTAL | wx.ALL, 5)
        self.button.Bind(wx.EVT_BUTTON, self.on_blank)
        
        main_sizer.Add((-1, 20))

        #Add End button
        self.end_process_button = wx.Button(panel, label = 'End Process')
        main_sizer.Add(self.end_process_button, 0, wx.ALIGN_CENTER_HORIZONTAL | wx.ALL, 5)
        self.end_process_button.Bind(wx.EVT_BUTTON, self.end_button)

        panel.SetSizer(main_sizer)
        self.Show(True)
    
    def get_file_path(self, event):
        global file_name
        wildcard = "Excel files (*.xls;*xlsx;)|*.xls;*.xlsx"
        dialog = wx.FileDialog(self, "Choose the Raw-data file", wildcard = wildcard, style = wx.FD_OPEN | wx.FD_FILE_MUST_EXIST)
        if dialog.ShowModal() == wx.ID_OK:
            print('Good')
            file_name = dialog.GetFilename()
            print(file_name)
        return file_name

    def onWellClick(self, event):
        well_button = event.GetEventObject()
        well_label = well_button.GetLabel()
        if well_label not in wells_clicked:
            wells_clicked.append(well_label)
        well_colour = well_button.GetBackgroundColour()
        if well_colour == wx.Colour(255, 255, 255) and self.status_button.GetValue() == 'Non-evolved':
            well_button.SetBackgroundColour(wx.Colour(252, 157, 3))
        elif well_colour == wx.Colour(255, 255, 255) and self.status_button.GetValue() == 'Evolved':
            well_button.SetBackgroundColour(wx.Colour(153, 153, 255))
        elif well_colour == wx.Colour(255, 255, 255) and self.status_button.GetValue() == 'BLANK':
            well_button.SetBackgroundColour(wx.Colour(255, 192, 203))
        else:
            well_button.SetBackgroundColour(wx.Colour(255, 255, 255))

    def on_add(self, event):
        global user_input
        global file_name
        global strain
        media = self.media_button.GetValue()
        status = self.status_button.GetValue()
        strain = self.strain_text.GetValue()
        inoc = self.inoc_button.GetValue()
        wells = wells_clicked.copy()
        data = {
                'Strain': strain,
                'Status': status,
                'Medium': media,
                'Wells': wells,
                'Inoculation': inoc
            }
        if unique_key in key_wells:
            key_wells[unique_key].append(data)
        else:
            key_wells[unique_key] = [data]
        wells_clicked.clear()
        #Check if strain is already in dictionary, overcoming overwriting of values
        if len(key_list) == 0:
            key_list[unique_key] = strain
        else:
            key_list[unique_key] = (new_keys)
        #New data appending with dataframe (several new parameters inculded):
        user_values = {'Strain': strain, 'Wells': wells, 'Media': media, 'Status': (status_list[-1]), 'Inoculation':inoc}
        appendix = pd.DataFrame([user_values])
        user_input = pd.concat([user_input, appendix], ignore_index= False)
        print('Added successfully')

    def on_blank(self, event):
        global user_input
        global blank
        global file_name
        media = self.media_button.GetValue()
        strain = self.strain_text.GetValue()
        wells = wells_clicked.copy()
        wells = wells_clicked.copy()
        wells_clicked.clear()
        data = {
                'Strain': strain,
                'Medium': media,
                'Wells': wells
            }
        blank = True
        if file_name in blank_dict:
            blank_dict[file_name].append(data)
        else:
            blank_dict[file_name] = [data]
        blank_measurements.extend(wells)
        
    def on_media_selection(self,event):
        selected_media = self.media_button.GetValue()
        media_selection.append(selected_media)

    def when_evolved(self, event):
        status= self.status_button.GetValue()
        if status == 'Evolved':
            status_list.append('Evolved')
        else:
            status_list.append('Non-evolved')

    def on_add_button_click(self, event):
        self.when_evolved(event)
        self.on_media_selection(event)
        self.on_add(event)

    def end_button(self, event):
        self.Close()


if __name__ == '__main__':
    app = wx.App()
    PlateFrame(None, title='96 Well Plate Strain Mapping')
    app.MainLoop()

print(file_name)
print(user_input)
print(key_list)

In [ ]:
full_json = {}
full_key_list = {}
try:
    with open('key_list.json', 'r') as json_file:
        existing_data = json.load(json_file)
except FileNotFoundError:
    existing_data = {}
existing_data.update(key_wells)
print(existing_data)
with open('key_list.json', 'w') as json_file:
    json.dump(existing_data, json_file, indent = 4)


In [ ]:
#Variables for the extraction of the right data
start_sheet = 32
end_sheet = 225

In [ ]:
df_specific = pd.read_excel(file_name, skiprows=start_sheet-1)
df_specific.dropna(inplace=True)
df_test_1 = df_specific
df_specific = df_specific.transpose()
df_specific.drop(index=df_specific.index[0], axis=0, inplace=True)
df_specific =df_specific.drop(columns=df_specific.columns[0])
df_specific

In [ ]:
df_test_1 = df_test_1.tail(-1)
df_test_1.insert(0, "Index", range(1,193))
df_test_1.reset_index(drop = True, inplace =True)
df_test_1.set_index("Index")

In [ ]:
def calculate_mean_blank(blank_dict, df):
    blank_sheet = {}
    for medium, items in blank_dict.items():
        for item in items:
            strain = item['Strain']
            medium = item['Medium']
            wells = item['Wells']
            blank_list = []
            for well in wells:
                row, column = np.where(df == well)
                if len(row) > 0:
                    well_index = int(row[0])
                    blank_list.extend(df.iloc[well_index, 2:].tolist())
                else:
                    print(f"Warning: Well {well} not found in DataFrame.")
            if blank_list:
                mean_blank = statistics.mean(blank_list)
                blank_sheet[medium] = mean_blank
    return blank_sheet

if blank == True:
    total_mean_blank = (calculate_mean_blank(blank_dict, df_test_1))


In [ ]:
def extraction(df_test_1, df, user_input, blank_dict, blank):
    b_list = []
    name_list = []
    well_list = []
    time_list = []
    evolution_list = []
    media_list = []
    for index, row in user_input.iterrows():
        strain = row['Strain']
        wells = []
        media = row['Media']
        status = row['Status']
        count = 0
        for well in row['Wells']:
            count += 1
            bacteria_list = []
            n_list = []
            well_data = []
            time_data = []
            row, column = np.where(df_test_1==well)
            well_index = int(row[0])
            well_data = (df_test_1.loc[well_index]).tolist()[2:]
            if blank == True:
                blank_value = blank_dict[media]
                well_data = [x - blank_value for x in well_data]
            time_data = (df_test_1.loc[well_index + 1]).tolist()[2:]
            time_data = [item/3600000 if item != 0 else item for item in time_data]
            bacteria_list.extend([strain]*len(well_data))
            n_list = ([well]*len(well_data))
            b_list.extend((bacteria_list))
            name_list.extend((n_list))
            well_list.extend(well_data)
            time_list.extend(time_data)
        media_list.extend([media] *(count * len(time_data)))
        evolution_list.extend([status]*(count * len(time_data)))
    df['Bacterial_strain'] = b_list
    df['Wells'] = name_list
    df['Extinction OD620nm'] = well_list
    df['Time in h'] = time_list
    df['Status'] = evolution_list
    df['Medium'] = media_list
    return df
DF_sheet = pd.DataFrame(columns=['Bacterial_strain', 'Medium', 'Status', 'Wells', 'Extinction OD620nm', 'Time in h'])
DF_sheet = extraction(df_test_1, DF_sheet, user_input, total_mean_blank, blank)

In [ ]:
pd.reset_option('all')
DF_sheet

In [ ]:
DF_sheet.to_csv('Datasheet ' + strain + ' ' + file_name, index = False, encoding= 'utf-8')

In [ ]:
g = sns.FacetGrid(DF_sheet, col='Medium', hue='Wells', aspect=2, col_wrap=2)
g.map(sns.lineplot, 'Time in h', 'Extinction OD620nm')
g.add_legend()  # Add legend here
plt.show()